# load Microsoft Project data from CSV

Successors and Predecessors use the following codes:
* FS = finish to start
* SS = start to start
* SF = start to finish

SF is not used very often

"+12d" = successor task will start 12 days after predecessor is completed

"-12d" = negative lag

In [1]:
import pandas
import matplotlib.pyplot as plt
import datetime
import random
import time
import sys
import pickle

In [2]:
df = pandas.read_csv("MS Project export.csv")

In [3]:
df

,ID,Duration,Start_Date,Finish_Date,Predecessors,Successors
0,1,5 days,"January 1, 2022 8:00 AM","January 1, 2022 8:00 AM",NaN,"2FF+23d,3SF+8d"
1,2,12 days,"May 21, 2022 5:00 PM","May 21, 2022 5:00 PM",5,4
2,3,6 days,"April 1, 2022 5:00 PM","April 1, 2022 5:00 PM",4FS+20d,NaN
3,4,10 days,"April 2, 2021 3:00 PM","April 2, 2021 3:00 PM",NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ID                          5 non-null      int64   
 1   Duration in days            5 non-null      int64   
 2   predecessor ID              2 non-null      object  
 3   successor ID                3 non-null      object  
 4   predecessor category        1 non-null      category
 5   successor category          1 non-null      category
 6   predecessor change in days  1 non-null      float64 
 7   successor change in days    2 non-null      float64 
dtypes: category(2), float64(2), int64(2), object(2)
memory usage: 522.0+ bytes


In [4]:
# df.assign(Predecessors_as_list=df["Predecessors"].str.split(","))

df["Predecessors as list"]=df["Predecessors"].str.split(",")
df["Successors as list"]=df["Successors"].str.split(",")

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df=df.drop(columns=["Predecessors","Successors","Start_Date","Finish_Date"])

df

,ID,Duration,Predecessors as list,Successors as list
0,1,5 days,NaN,"[2FF+23d, 3SF+8d]"
1,2,12 days,[5],[4]
2,3,6 days,[4FS+20d],NaN
3,4,10 days,NaN,NaN


In [5]:
# https://cmdlinetips.com/2020/06/pandas-explode-convert-list-like-column-elements-to-separate-rows/
df = df.explode("Successors as list")
df = df.explode("Predecessors as list")
df

,ID,Duration,Predecessors as list,Successors as list
0,1,5 days,NaN,2FF+23d
0,1,5 days,NaN,3SF+8d
1,2,12 days,5,4
2,3,6 days,4FS+20d,NaN
3,4,10 days,NaN,NaN


In [6]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
df = df.rename(columns={"Successors as list":"Successor", 
                        "Predecessors as list": "Predecessor",
                        "Duration": "Duration in days"})
df

,ID,Duration in days,Predecessor,Successor
0,1,5 days,NaN,2FF+23d
0,1,5 days,NaN,3SF+8d
1,2,12 days,5,4
2,3,6 days,4FS+20d,NaN
3,4,10 days,NaN,NaN


In [7]:
# https://stackoverflow.com/a/59652937/1164295
# https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html

df["predecessor ID"] = df["Predecessor"].str.extract(r'(\d+)')[0]
df["successor ID"] = df["Successor"].str.extract(r'(\d+)')[0]

df["predecessor category"] = df["Predecessor"].str.extract("(FS|SS|SF)")[0]
df["successor category"] = df["Successor"].str.extract("(FS|SS|SF)")[0]

df["predecessor change in days"] = df["Predecessor"].str.extract(r'(\+\d+d|-\d+d)')
df["successor change in days"] = df["Successor"].str.extract(r'(\+\d+d|-\d+d)')

df["predecessor category"] = (df["predecessor category"]
 .str.replace("FS", "finish-to-start")
 .str.replace("SF", "start-to-finish")
 .str.replace("SS", "start-to-start")
)
df["successor category"] = (df["successor category"]
 .str.replace("FS", "finish-to-start")
 .str.replace("SF", "start-to-finish")
 .str.replace("SS", "start-to-start")
)

df

,ID,Duration in days,Predecessor,Successor,predecessor ID,successor ID,predecessor category,successor category,predecessor change in days,successor change in days
0,1,5 days,NaN,2FF+23d,NaN,2,NaN,NaN,NaN,+23d
0,1,5 days,NaN,3SF+8d,NaN,3,NaN,start-to-finish,NaN,+8d
1,2,12 days,5,4,5,4,NaN,NaN,NaN,NaN
2,3,6 days,4FS+20d,NaN,4,NaN,finish-to-start,NaN,+20d,NaN
3,4,10 days,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df=df.drop(columns=["Predecessor","Successor"])

df

,ID,Duration in days,predecessor ID,successor ID,predecessor category,successor category,predecessor change in days,successor change in days
0,1,5 days,NaN,2,NaN,NaN,NaN,+23d
0,1,5 days,NaN,3,NaN,start-to-finish,NaN,+8d
1,2,12 days,5,4,NaN,NaN,NaN,NaN
2,3,6 days,4,NaN,finish-to-start,NaN,+20d,NaN
3,4,10 days,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# https://stackoverflow.com/a/66607872/1164295

df["Duration in days"]=df["Duration in days"].str.replace(" days","", regex=False)
df["predecessor change in days"]=df["predecessor change in days"].str.replace("d","", regex=False)
df["predecessor change in days"]=df["predecessor change in days"].str.replace("+","", regex=False)
df["successor change in days"]=df["successor change in days"].str.replace("d","", regex=False)
df["successor change in days"]=df["successor change in days"].str.replace("+","", regex=False)
df

,ID,Duration in days,predecessor ID,successor ID,predecessor category,successor category,predecessor change in days,successor change in days
0,1,5,NaN,2,NaN,NaN,NaN,23
0,1,5,NaN,3,NaN,start-to-finish,NaN,8
1,2,12,5,4,NaN,NaN,NaN,NaN
2,3,6,4,NaN,finish-to-start,NaN,20,NaN
3,4,10,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          5 non-null      int64 
 1   Duration in days            5 non-null      object
 2   predecessor ID              2 non-null      object
 3   successor ID                3 non-null      object
 4   predecessor category        1 non-null      object
 5   successor category          1 non-null      object
 6   predecessor change in days  1 non-null      object
 7   successor change in days    2 non-null      object
dtypes: int64(1), object(7)
memory usage: 360.0+ bytes


In [11]:
df["Duration in days"] = pandas.to_numeric(df["Duration in days"])
# leave these as strings; there's no point to having numeric IDs
#df["predecessor ID"] = pandas.to_numeric(df["predecessor ID"])
#df["successor ID"] = pandas.to_numeric(df["successor ID"])

# https://stackoverflow.com/a/28648923/1164295
df["predecessor change in days"] = pandas.to_numeric(df["predecessor change in days"])
df["successor change in days"] = pandas.to_numeric(df["successor change in days"])

# https://pandas.pydata.org/docs/user_guide/categorical.html
df["predecessor category"]= df["predecessor category"].astype("category")
df["successor category"]= df["successor category"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 3
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ID                          5 non-null      int64   
 1   Duration in days            5 non-null      int64   
 2   predecessor ID              2 non-null      object  
 3   successor ID                3 non-null      object  
 4   predecessor category        1 non-null      category
 5   successor category          1 non-null      category
 6   predecessor change in days  1 non-null      float64 
 7   successor change in days    2 non-null      float64 
dtypes: category(2), float64(2), int64(2), object(2)
memory usage: 522.0+ bytes


one last visual check

In [12]:
df

,ID,Duration in days,predecessor ID,successor ID,predecessor category,successor category,predecessor change in days,successor change in days
0,1,5,NaN,2,NaN,NaN,NaN,23.0
0,1,5,NaN,3,NaN,start-to-finish,NaN,8.0
1,2,12,5,4,NaN,NaN,NaN,NaN
2,3,6,4,NaN,finish-to-start,NaN,20.0,NaN
3,4,10,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# https://wiki.python.org/moin/UsingPickle

with open("project.pkl", "wb") as file_handle:
    pickle.dump( df, file_handle )